In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery

# BigQueryクライアントの初期化
client = bigquery.Client(project='rd-dapj-dev')

# 日付ごとのデータを取得するクエリ
dates = ["PDP_20211007", "PDP_20211008", "PDP_20231015", 
         "PDP_20231016", "PDP_20231023", "PDP_20231029"]

# 結果を格納するデータフレームのリスト
dfs = []

for date in dates:
    query = f"""
    SELECT 
        stay_duration,
        FORMAT_DATE('%Y-%m-%d', DATE(visit_time)) as visit_date,
        uuid
    FROM `rd-dapj-dev.raw_daimaruyu_data.{date}`
    WHERE visit_style = 'worker'
    """
    df = client.query(query).to_dataframe()
    dfs.append(df)

# 全データを結合
all_data = pd.concat(dfs)

# 滞在時間の最大値と最小値を取得（軸の範囲設定用）
max_duration = all_data['stay_duration'].max()
min_duration = all_data['stay_duration'].min()

# プロットの設定
plt.style.use('seaborn')
fig, axes = plt.subplots(2, 3, figsize=(20, 12))
axes = axes.ravel()

# 日付ごとの分布図を作成
for idx, date in enumerate(dates):
    date_data = all_data[all_data['visit_date'] == date]
    unique_users = len(date_data['uuid'].unique())
    
    sns.histplot(
        data=date_data,
        x='stay_duration',
        bins=30,
        ax=axes[idx]
    )
    
    # 軸の範囲を統一
    axes[idx].set_xlim(min_duration, max_duration)
    axes[idx].set_ylim(0, axes[0].get_ylim()[1])  # 最初のプロットの最大値に合わせる
    
    axes[idx].set_title(f'滞在時間分布 ({date})\nユーザー数: {unique_users}名', pad=10)
    axes[idx].set_xlabel('滞在時間 (分)')
    axes[idx].set_ylabel('頻度')

plt.tight_layout()
plt.suptitle('ワーカーの滞在時間分布（日付別）', y=1.02, fontsize=16)
plt.show()

# 基本統計量の出力
print("\n各日付の滞在時間の基本統計量:")
for date in dates:
    date_data = all_data[all_data['visit_date'] == date]
    print(f"\n{date} (ユーザー数: {len(date_data['uuid'].unique())}名):")
    print(date_data['stay_duration'].describe())